In [ ]:
import io, os, sys
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

import collections
INF = 1 << 30
class MCMF:
  def __init__(self, n) :
    self.len = n
    self.G = [[] for _ in range(n)]
    self.C = [[0] * n for _ in range(n)]
    self.F = [[0] * n for _ in range(n)]
    self.W = [[0] * n for _ in range(n)]

  def addEdge(self, u, v, c, w) :
    self.G[u].append(v)
    self.G[v].append(u)
    self.C[u][v] = c
    self.W[u][v] = w
    self.W[v][u] = -w

  def SPFA(self, src, snk):
    mc = mf = 0
    while True :
      vis = [False] * self.len #해당 정점이 큐 안에 있는지 확인
      dist = [INF] * self.len #해당 정점까지의 최소 거리
      par = [-1] * self.len #해당 정점으로 가기 직전에 거쳐온 정점

      Q = collections.deque([src])
      dist[src] = 0
      vis[src] = True
      while Q :
        u = Q.popleft()
        vis[u] = False
        for v in self.G[u] :
          if self.C[u][v] - self.F[u][v] > 0 and dist[v] > dist[u] + self.W[u][v] : #Residual Capacity가 남아있고, 최소 거리를 갱신할 수 있으면
            dist[v] = dist[u] + self.W[u][v]
            par[v] = u
            if not vis[v] : #큐에 들어있지 않으면 큐에 넣는다
              vis[v] = True
              Q.append(v)

      if par[snk] == -1 : break #더이상 Augment Path가 없으면 종료

      flow = INF
      u = snk
      while u != src : #Augment Path를 통해 얻을 수 있는 최대 유량 계산
        flow = min(flow, self.C[par[u]][u] - self.F[par[u]][u])
        u = par[u]
      
      u = snk
      while u != src : #경로상의 모든 간선에 유량을 흘린다.
        mc += flow * self.W[par[u]][u]
        self.F[par[u]][u] += flow
        self.F[u][par[u]] -= flow
        u = par[u]

      mf += 1

    return mc, mf
    
def sol(): 
  N, M = map(int, input().split())
  mcmf = MCMF(N+M+2)
  src, snk = N+M, N+M+1

  for i in range(N) :
    n, *l = map(int, input().split())
    mcmf.addEdge(src, i, 1, 0) #src와 직원 매칭
    for j, w in zip(l[::2], l[1::2]) :
      mcmf.addEdge(i, N+j-1, n, w) #직원과 일 매칭
  
  for i in range(M) :
    mcmf.addEdge(N+i, snk, 1, 0)

  mc, mf = mcmf.SPFA(src, snk)     
  sys.stdout.write(f'{mf}\n{mc}')
  
sol()

### 풀이
- MCMF 문제. 가장 쉬워보이는 SPFA를 구현했다.
- 각 직원은 일을 하나씩 할 수 있으므로, src에서 직원의 용량을 1, 일도 직원 한명만 작업하면 되므로 일에서 sink까지의 용량도 1로 설정한다.
  - 이때, src와 sink간의 연결에서 비용이 발생하진 않으므로 그 구간의 비용은 0이다.
- 어떤 직원이 할 수 있는 일과 매칭할 때 비용이 발생하므로 입력에서 주워진 비용을 적용한다 `mcmf.addEdge(i, N+j-1, n, w)`
  - 이때 용량은 최대 `직원이 할 수 있는 일의 개수`까지만 할당할 수 있으므로 `n` 이상으로 할당하면 된다.